In [6]:
import open3d
import numpy as np
import pandas as pd
import os
import glob
import tqdm
import copy

import utils.registration as registration
import utils.fread as fread
import utils.transform as transform
import utils.grid_search as grid_search
import utils.functions as functions
from utils.depth_camera import DepthCamera

from time import sleep
from PIL import Image
from utils.config import Config

In [7]:
depth_camera = DepthCamera("secondary-camera", "data/raw_data/exp_12/metadata/device-3-aligned.json")

In [3]:
config = Config(
    sequence_dir="data/raw_data",
    feature_dir="data/features",
    output_dir="data/trajectories/temp",
    experiment="exp_12",
    trial="trial_1",
    subject="subject-1",
    sequence="01",
    groundtruth_dir="data/trajectories/groundtruth",
)
    
config.voxel_size=0.05
config.target_fps=20
config.min_std=0.5

In [5]:
feature_dir = config.get_feature_dir()
sequence_dir = config.get_sequence_dir()

sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)

num_frames = len(sequence_ts)
print(f"Number of frames: {num_frames}")

Number of frames: 450


In [11]:
local_pcds = []
fpfh_feats = []

for t in tqdm.trange(num_frames):
    depth_img_file = os.path.join(sequence_dir, f"frame-{sequence_ts[t]}.depth.png")
    pcd = depth_camera.depth_to_point_cloud(depth_img_file)
    pcd = open3d.voxel_down_sample(pcd, voxel_size=config.voxel_size)
    pcd, fpfh = registration.compute_fpfh(pcd, config.voxel_size, down_sample=False)
    local_pcds.append(pcd)
    fpfh_feats.append(fpfh)

100%|██████████| 450/450 [00:18<00:00, 24.62it/s]


In [ ]:
local_pcds = []
fpfh_feats = []

start_time = time.time()

for t in tqdm.trange(num_frames):
    feature_file = os.path.join(feature_dir, f"{sequence_ts[t]}.secondary.npz")
    pcd = FCGF.get_features(feature_file, config.voxel_size, pcd_only=True)
    pcd, fpfh = registration.compute_fpfh(pcd, config.voxel_size, down_sample=False)
    local_pcds.append(pcd)
    fpfh_feats.append(fpfh) 